# MosquitoAlert - YoloV5 Baseline Submission

![](https://images.aicrowd.com/raw_images/challenges/social_media_image_file/1124/1c83e8678106d51ab51f.png)

This notebook will contains instructions and example submission with yolov5 trained on the dataset.

## Installations 🤖

1. `aicrowd-cli` for downloading challenge data and making submissions
2. `pyarrow` for saving to parquet for submissions

In [2]:
!pip install -U --user ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

^C


In [4]:
!pip install aicrowd-cli

You should consider upgrading via the 'C:\Users\dinle\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import torch
import os
from pathlib import Path
import cv2
from timeit import default_timer as timer
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

# Download Dataset and Models

In [6]:
!aicrowd login

'aicrowd'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [ ]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023

In [ ]:
!unzip -qq phase2_train_v0.zip -d phase2_train_v0/

In [ ]:
# Downloading the weight file at https://drive.google.com/file/d/1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS/view
!gdown "1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS"

In [ ]:
TEST_DATA_DIR = 'phase2_train_v0/'

In [ ]:
def classify_image(image):
    image_information = {}
    result = yolov5_model(image)
    result_df = result.pandas().xyxy[0]
    if result_df.empty:
        print('No results from yolov5 model!')
    else:
        image_information = result_df.to_dict()
    return image_information

# getting mosquito_class name from predicted result
def extract_predicted_mosquito_class_name(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("name").get(0))
    return mosquito_class

# getting mosquito_class number from predicted result
def extract_predicted_mosquito_class_number(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("class").get(0))
    return mosquito_class

# getting mosquito_class confidence score from predicted result
def extract_predicted_mosquito_class_confidence(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("confidence").get(0))
    return mosquito_class

# getting mosquito bounding box from predicted result
# it looks like the results are in different notation
# than the uoutput file
# Pascal VOC? (top, left, bottom, right)?
def extract_predicted_mosquito_bbox(extractedInformation):
    bbox = []
    if extractedInformation is not None:
        xmin = str(extractedInformation.get("xmin").get(0))
        ymin = str(extractedInformation.get("ymin").get(0))
        xmax = str(extractedInformation.get("xmax").get(0))
        ymax = str(extractedInformation.get("ymax").get(0))
        bbox = [xmin, ymin, xmax, ymax]
    return bbox

# find image id
def find_image_id(original_image):
    image_name = os.path.splitext(original_image)[0]
    return image_name

In [ ]:
# path to dataset
root_images = os.path.join(TEST_DATA_DIR)

all_images = os.listdir(root_images)
print(f"Total images: {len(all_images)}")

class_labels = {
    "aegypti":      0,
    "albopictus":   1,
    "anopheles":    2,
    "culex":        3,
    "culiseta":     4,
    "japonicus/koreicus": 5
}

# counter for correctly recognized mosquito classes
counter = 0
labels_counter = 0
no_mosquito_detected = 0
rows = []

# yolov5 challenge trained baseline
trained_model_pretrained = "mosquitoalert-yolov5-baseline.pt"

# yolov5_model
# Colab gets an error on this -> Use Runtime -> Restart Session (The data will not be deleted)
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path = trained_model_pretrained, force_reload = True)

for original_image in tqdm(all_images):
    try:
        original_image_file = os.path.join(root_images, original_image)
        # checking if it is a file
        if os.path.isfile(original_image_file):
            # opening testing image
            # print(f'You are watching: {original_image}')
            # classifying image by yolov5 model
            predictedInformation = classify_image(original_image_file)
            mosquito_class_name_predicted = ""
            mosquito_class_number_predicted = ""
            mosquito_class_confidence = 0
            mosquito_class_bbox = [0, 0, 0, 0]

            if predictedInformation:
                mosquito_class_name_predicted = extract_predicted_mosquito_class_name(predictedInformation)
                mosquito_class_number_predicted = extract_predicted_mosquito_class_number(predictedInformation)
                mosquito_class_confidence = extract_predicted_mosquito_class_confidence(predictedInformation)
                mosquito_class_bbox = extract_predicted_mosquito_bbox(predictedInformation)
                counter += 1
            else:
                no_mosquito_detected += 1
            # print(f"Predicted mosquito class: {mosquito_class_name_predicted} with {float(mosquito_class_confidence):.2f} confidence score.")
            #  bbox = [xmin, ymin, xmax, ymax]
            # create row for csv file
            row = [original_image, mosquito_class_name_predicted, mosquito_class_number_predicted, mosquito_class_confidence,
                   mosquito_class_bbox[0], mosquito_class_bbox[1], mosquito_class_bbox[2], mosquito_class_bbox[3]]
            rows.append(row)
            labels_counter += 1
            # print(f"Finished file reading, file {original_image} read correctly!")
    except Exception as e:
        print(f'Unable to process file: {original_image}!')
        print(f'Exception: {e}!')

In [ ]:
df = pd.DataFrame(rows, columns=["img_fName", "class_label", "predicted_class_number", "confidence_score", "bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr"])
sub_df = df.drop(['predicted_class_number', 'confidence_score'], axis=1, inplace=False)
sub_df.head()

In [ ]:
sub_df.to_csv('phase2_sub.csv', index=False)

In [ ]:
!aicrowd submission create -c mosquitoalert-challenge-2023 -f "phase2_sub.csv"